<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/Q%26A_ChatBot2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import os
import tempfile
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, Runnable
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages

# 환경 변수 불러오기(openai API 키)
load_dotenv('/content/drive/MyDrive/Colab Notebooks/.env')

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini")

# 텍스트 분리
text_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100
)

# 임베딩 모델
hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 프롬프트 템플릿
system_message = """
당신은 사용자의 질문에 답변을 하는 친절한 AI 어시스턴트입니다.
당신의 임무는 주어진 문맥을 토대로 사용자 질문에 답하는 것입니다.
만약, 문맥에서 답변을 위한 정보를 찾을 수 없다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다' 라고 답하세요.
정보를 찾을 수 있다면 한글로 답변해 주세요.
"""

human_message = """
## 과거 대화 이력:
{history}

## 검색된 문서:
{context}

## 최신 사용자 질문:
{input}
"""
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", human_message),
    ]
)

# 출력 파서
parser = StrOutputParser()

# 트리머 설정
trimmer = trim_messages(
    max_tokens=1000,
    token_counter=llm,
    strategy="last",
    include_system=True,
    start_on="human"
)

# 전역 변수
db = None
retriever = None
rag_chain = None
faiss_path = "/content/drive/MyDrive/FaissDB/knu_faiss_db"
history_store = {}  # 메시지 이력 저장용 전역 스토어

# 세션 히스토리 함수 정의
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in history_store:
        messages = InMemoryChatMessageHistory()
        # ② 생성 직후에 시스템 메시지를 한 번 저장
        messages.add_message(SystemMessage(content=system_message))
        history_store[session_id] = messages
    return history_store[session_id]

# RAG 체인 구성, 사용자 질문에 대한 응답을 만들기
def setup_chain():
    global retriever, rag_chain

    # 과거 대화 이력만 꺼내오는 체인 history
    history = RunnableWithMessageHistory(
        RunnablePassthrough(),
        get_session_history
    )
    trimmed_history = trimmer | history

    # 검색기 context
    retriever = db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "fetch_k": 10, "lamda_mult": 0.5}
    )

    # 최종 RAG 체인
    rag_chain = {
        "history": trimmed_history,
        "context": retriever,
        "input": RunnablePassthrough()
    } | prompt_template | llm | parser

# 드라이브에 존재하는 DB 로드
def load_faiss_db():
    global db
    db = FAISS.load_local(
        folder_path=faiss_path,
        embeddings=hf_embeddings,
        allow_dangerous_deserialization=True
    )
    setup_chain()

# PDF 업로드 및 DB 저장
def add_pdf_to_db(file):
    global db

    loader = PyPDFLoader(file.name)
    docs = loader.load_and_split(text_splitter=text_splitter)

    # 각 청크에 파일명 metadata 추가
    for doc in docs:
        doc.metadata["file_name"] = os.path.basename(file.name)
        # 여기에 url도 넣을 수 있을듯?

    if db is None:
        db = FAISS.from_documents(docs, hf_embeddings)
    else:
      db.add_documents(docs)

    db.save_local(faiss_path)
    setup_chain()

    return f"{os.path.basename(file.name)} 문서를 처리하여 FAISS DB에 저장했습니다."

# 질문 처리
def answer_question(question):
    if rag_chain is None:
        return "먼저 PDF 파일을 업로드하세요!"
    # 1) 세션 이력 객체 꺼내기
    messages = get_session_history("chat")
    # 2) HumanMessage 형태로 질문 저장
    messages.add_message(HumanMessage(content=question))
    # 3) RAG 체인 호출 → 답변 생성
    answer = rag_chain.invoke(
        question,
        config={"configurable": {"session_id": "chat"}}
    )
    # 4) AIMessage 형태로 답변 저장
    messages.add_message(AIMessage(content=answer))
    return answer

# 저장된 문서 목록 표시
def show_stored_documents():
    if db is None:
        return "DB 로드 문제"
    docs = list(db.docstore._dict.values())  # 저장된 모든 청크들을 가져와 리스트로 변환
    file_names = {doc.metadata.get("file_name", "Unknown") for doc in docs}
    return "📚 저장된 문서 목록:\n" + "\n".join(f"• {f}" for f in sorted(file_names))

# Gradio UI 설정
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📄 인공지능 PDF Q&A 챗봇
    **여러 PDF 파일을 업로드하고 질문을 입력하면 AI가 답변을 제공합니다!**
    """)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="PDF 파일 선택")
            upload_button = gr.Button("📤 벡터 DB에 저장")
            show_files_button = gr.Button("📚 저장된 문서 보기")
            status_output = gr.Textbox(label="📢 상태 메시지")

        with gr.Column(scale=2):
            question_input = gr.Textbox(label="❓ 질문 입력", placeholder="궁금한 내용을 적어주세요.")
            submit_button = gr.Button("🤖 답변 받기")
            answer_output = gr.Textbox(label="📝 AI 답변")

    upload_button.click(add_pdf_to_db, inputs=file_input, outputs=status_output)
    submit_button.click(answer_question, inputs=question_input, outputs=answer_output)
    show_files_button.click(show_stored_documents, outputs=status_output)

# 벡터 DB 로드 후 실행
load_faiss_db()
demo.launch()